In [1]:
import sifce
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical

In [2]:
# get ifos
ifos=['H1', 'L1', 'V1']
ifos_det = dict()

for ifo in ifos:
    ifos_det[ifo] = Detector(ifo)

In [3]:
from sifce import population
from sifce import datatools

# samples for generating waves
prior = dict(mass_1= DeltaFunction(35),
             mass_2= DeltaFunction(35),
             a_1 = DeltaFunction(0.5),
             a_2 = DeltaFunction(0.5),
             tilt_1 = DeltaFunction(1.4),
             tilt_2 = DeltaFunction(1.4),
             phi_12 = DeltaFunction(3.09),
             phi_jl = DeltaFunction(5.21),
             theta_jn =  Sine(name='theta_jn'),
             phase = DeltaFunction(1.04),
             reference_frequency = DeltaFunction(20))
prior_obj = PriorDict(dictionary = prior)
pop = population.SimulationSet(distribution=prior_obj)
pop.sample_distribution(100)

In [4]:
# sky samples
prior_sky = dict(dec=Cosine(name='dec'),
                ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),)
prior_sky_obj = PriorDict(dictionary = prior_sky)
sky_pop = prior_sky_obj.sample(1000)

In [5]:
# set psd
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

psd_dict = {'H1': psd, 'L1': psd, 'V1':psd}
apx='IMRPhenomXPHM'

In [6]:
# calculate snr
sky_catter_results_list = list()

temp_pd = pop.simulations_dataframe

for index, row in temp_pd.iterrows():
    tmp_dict = row.to_dict()
    
    # get hp and hc:
    param_args = { k: tmp_dict[k] for k in pop.waveform_generation_labels}
    strain = dict()
    strain['plus'], strain['cross'] = datatools.compute_hphc_fd(param_args, apx)
        
    #sky scatter:
    opt_snr_arrays = datatools.sky_scatter_snr(sky_pop, {'plus_template': strain['plus'], 'cross_template': strain['cross']}, 0, psd_dict, ifo_list=ifos)
    sky_catter_results_list.append(opt_snr_arrays)
    

/home/daniela.yano/summer-2022/sifce/sifce/datatools.py:410: RuntimeWarning: overflow encountered in square
  rho_mf_net += rho_mf_array**2


In [7]:
# need to rearange dataframe
df= pop.simulations_dataframe
df= pd.concat(1000*[pop.simulations_dataframe])
df= df.sort_index(ascending=True)
df=df.reset_index()

In [8]:
df_sky=pd.DataFrame(sky_pop)
df_sky=pd.concat(100*[df_sky])
df_sky=df_sky.reset_index()

In [9]:
df['ra']=df_sky['ra']
df['dec']=df_sky['dec']
df['psi']=df_sky['psi']

In [10]:
pd.set_option('display.max_columns', None)
df.head(1000)

,index,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,4.749924,-1.171978,2.089812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
1,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.533187,-1.006192,2.998786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
2,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.217143,1.017393,1.497115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
3,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.247407,-0.471342,2.165964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
4,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.606852,0.077067,0.588369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.072789,0.044603,2.085990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
996,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.352015,0.036731,2.574502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
997,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.222775,0.907761,1.849879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177
998,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.042524,-0.783446,0.209788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.304424,5.757362,2.564177


In [11]:
df.tail(1000)

,index,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
99000,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,4.749924,-1.171978,2.089812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99001,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,2.533187,-1.006192,2.998786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99002,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.217143,1.017393,1.497115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99003,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,1.247407,-0.471342,2.165964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99004,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,5.606852,0.077067,0.588369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.072789,0.044603,2.085990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99996,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.352015,0.036731,2.574502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99997,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.222775,0.907761,1.849879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223
99998,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,2.042524,-0.783446,0.209788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,1.18711,5.765415,2.57223


In [12]:
# add opt_snr
tmp_snr_network_list = list()
for snr_tuple in sky_catter_results_list:
    tmp_snr_network_list.extend(snr_tuple[0]['net'])

In [13]:
df['opt_snr'] = tmp_snr_network_list

In [14]:
df.head(10)

,index,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,4.749924,-1.171978,2.089812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,50736.655263,NaN,0.304424,5.757362,2.564177
1,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.533187,-1.006192,2.998786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,52447.384199,NaN,0.304424,5.757362,2.564177
2,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.217143,1.017393,1.497115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,58315.521761,NaN,0.304424,5.757362,2.564177
3,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.247407,-0.471342,2.165964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,25414.714914,NaN,0.304424,5.757362,2.564177
4,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.606852,0.077067,0.588369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,47640.556297,NaN,0.304424,5.757362,2.564177
5,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,3.773144,0.095729,2.120762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,39433.749721,NaN,0.304424,5.757362,2.564177
6,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.142561,0.057196,2.847899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,46191.827229,NaN,0.304424,5.757362,2.564177
7,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,6.012145,0.918419,1.882670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,56163.717550,NaN,0.304424,5.757362,2.564177
8,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.686525,-0.360564,0.639930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,32555.829943,NaN,0.304424,5.757362,2.564177
9,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.246916,0.932962,1.283356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,46917.318874,NaN,0.304424,5.757362,2.564177


In [15]:
# now I need to generate luminosity distance
prior_ldistance = dict(luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))
prior_ldistanex_obj = PriorDict(dictionary = prior_ldistance)
lum_distribution = prior_ldistanex_obj.sample(1000)

In [ ]:
# do scaling and update opt_snr
df_ld=pd.DataFrame(lum_distribution)
df_ld=pd.concat(100*[df_ld])
df_ld=df_ld.reset_index()

df_ld.head(10)

In [17]:
df['opt_snr']=df['opt_snr']/df_ld['luminosity_distance']
df['luminosity_distance']=df_ld['luminosity_distance']

In [18]:
df.head(10)

,index,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,4.749924,-1.171978,2.089812,NaN,1719.796383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,29.501548,NaN,0.304424,5.757362,2.564177
1,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.533187,-1.006192,2.998786,NaN,1780.245195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,29.460764,NaN,0.304424,5.757362,2.564177
2,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.217143,1.017393,1.497115,NaN,1312.664103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,44.425319,NaN,0.304424,5.757362,2.564177
3,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.247407,-0.471342,2.165964,NaN,725.726082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,35.019707,NaN,0.304424,5.757362,2.564177
4,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.606852,0.077067,0.588369,NaN,1542.850887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,30.878264,NaN,0.304424,5.757362,2.564177
5,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,3.773144,0.095729,2.120762,NaN,1817.526742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,21.696379,NaN,0.304424,5.757362,2.564177
6,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.142561,0.057196,2.847899,NaN,1347.141593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,34.288769,NaN,0.304424,5.757362,2.564177
7,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,6.012145,0.918419,1.882670,NaN,1795.189855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,31.285670,NaN,0.304424,5.757362,2.564177
8,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.686525,-0.360564,0.639930,NaN,1133.384180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,28.724444,NaN,0.304424,5.757362,2.564177
9,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.246916,0.932962,1.283356,NaN,1994.742966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,23.520483,NaN,0.304424,5.757362,2.564177


In [19]:
# filter opt_snt > 11 
df_final = df[df['opt_snr']>=11]

In [23]:
df_final

,index,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,4.749924,-1.171978,2.089812,NaN,1719.796383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,29.501548,NaN,0.304424,5.757362,2.564177
1,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,2.533187,-1.006192,2.998786,NaN,1780.245195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,29.460764,NaN,0.304424,5.757362,2.564177
2,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,0.217143,1.017393,1.497115,NaN,1312.664103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,44.425319,NaN,0.304424,5.757362,2.564177
3,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,1.247407,-0.471342,2.165964,NaN,725.726082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,35.019707,NaN,0.304424,5.757362,2.564177
4,0,35.0,35.0,0.426163,-0.247311,0.084984,-0.412842,0.268959,0.084984,0.299077,1.04,5.606852,0.077067,0.588369,NaN,1542.850887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,30.878264,NaN,0.304424,5.757362,2.564177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.072789,0.044603,2.085990,NaN,1786.193250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,12.575605,NaN,1.187110,5.765415,2.572230
99996,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.352015,0.036731,2.574502,NaN,1218.129994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,21.592354,NaN,1.187110,5.765415,2.572230
99997,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,0.222775,0.907761,1.849879,NaN,1484.503505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,17.599513,NaN,1.187110,5.765415,2.572230
99998,99,35.0,35.0,0.428141,-0.243871,0.084984,-0.414995,0.265626,0.084984,1.181774,1.04,2.042524,-0.783446,0.209788,NaN,1518.398208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,13.069323,NaN,1.187110,5.765415,2.572230


In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80664 entries, 0 to 99999
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 80664 non-null  int64  
 1   mass_1                80664 non-null  float64
 2   mass_2                80664 non-null  float64
 3   spin_1x               80664 non-null  float64
 4   spin_1y               80664 non-null  float64
 5   spin_1z               80664 non-null  float64
 6   spin_2x               80664 non-null  float64
 7   spin_2y               80664 non-null  float64
 8   spin_2z               80664 non-null  float64
 9   theta_jn              80664 non-null  float64
 10  phase                 80664 non-null  float64
 11  ra                    80664 non-null  float64
 12  dec                   80664 non-null  float64
 13  psi                   80664 non-null  float64
 14  geocent_time          0 non-null      object 
 15  luminosity_distance